# v2.1 exploration trying to make it work better

In [ ]:
# Environment
import os
import os.path as osp
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
import sys
# Local modules
sys.path.append('..')
import reproducibility
import pandas as pd
from utils import print_dict_summary
from data_funcs import rmse
from moisture_rnn import RNNParams, RNNData, RNN, RNN_LSTM, create_rnn_data2
from moisture_rnn_pkl import pkl2train
from tensorflow.keras.callbacks import Callback
from utils import hash2
import copy
import logging
import pickle
from utils import logging_setup, read_yml, read_pkl, hash_ndarray, hash_weights
import yaml
import copy

In [ ]:
logging_setup()

## Tests

In [ ]:
# data_file = 'data/test_CA_202401.pkl'
# train = pkl2train([data_file])
# # with open("train.pkl", 'wb') as file:
# #     logging.info('Writing the rain cases into file %s',"train.pkl")
# #     pickle.dump(train, file)
# train.keys()

train = read_pkl("train.pkl")

In [ ]:
train.keys()

In [ ]:
case = "FBFW1_202401"
dat = train[case]

In [ ]:
params = read_yml("params.yaml", subkey="rnn")
params = RNNParams(params)
params

In [ ]:
rnn_dat = RNNData(dat, scaler = params['scaler'], features_list = params['features_list'])
rnn_dat.train_test_split(
    train_frac = .9,
    val_frac = .05
)
rnn_dat.scale_data()

In [ ]:
import importlib
import moisture_rnn
importlib.reload(moisture_rnn)
from moisture_rnn import RNN

In [ ]:
params.update({'epochs': 20})
reproducibility.set_seed(123)
rnn = RNN(params)


In [ ]:
m, errs = rnn.run_model(rnn_dat)

In [ ]:
rnn.model_train

## Test RNN

In [ ]:
import importlib
import utils
importlib.reload(utils)
from utils import read_pkl

In [ ]:
train = read_pkl("train.pkl")

In [ ]:
params = read_yml("params.yaml", subkey="rnn")
params = RNNParams(params)

In [ ]:
params.update({'activation': ['linear', 'linear'], 'epochs':300, 'val_frac': .2, 'scaler': 'minmax', 'rnn_layers': 1, 'dense_layers': 1})

In [ ]:
rnn_dat = RNNData(train['NV020_202401'], scaler = params['scaler'], features_list = params['features_list'])

In [ ]:
rnn_dat.train_test_split(
    train_frac = params['train_frac'],
    val_frac = params['val_frac']
)

In [ ]:
rnn_dat.scale_data()

In [ ]:
import importlib
import moisture_rnn
importlib.reload(moisture_rnn)
from moisture_rnn import RNN

In [ ]:
rnn = RNN(params)

In [ ]:
rnn.predict(rnn_dat.X_test)

In [ ]:
rnn.model_predict.summary()

In [ ]:
reproducibility.set_seed(123)
rnn = RNN(params)
m, errs = rnn.run_model(rnn_dat)

## Test Data Creation

In [ ]:
import importlib
import moisture_rnn_pkl
importlib.reload(moisture_rnn_pkl)
from moisture_rnn_pkl import pkl2train

In [ ]:
file_names=['test_CA_202401.pkl', 'test_NW_202401.pkl']
file_dir='data'
file_paths = [osp.join(file_dir,file_name) for file_name in file_names]

In [ ]:
train = pkl2train(file_paths)

In [ ]:
train.keys()

## Test RNN

In [ ]:
errs

In [ ]:
import importlib 
import moisture_rnn
importlib.reload(moisture_rnn)
from moisture_rnn import RNN

In [ ]:
params = read_yml("params.yaml", subkey="rnn")
params = RNNParams(params)

In [ ]:
reproducibility.set_seed()
rnn = RNN(params)
m, errs = rnn.run_model(rnn_dat)

## LSTM

TODO: FIX BELOW

In [ ]:
import importlib 
import moisture_rnn
importlib.reload(moisture_rnn)
from moisture_rnn import RNN_LSTM

In [ ]:
with open("params.yaml") as file:
    params = yaml.safe_load(file)["lstm"]
    
rnn_dat2 = create_rnn_data2(train[case],params)

In [ ]:
params.update({'epochs': 10})

In [ ]:
reproducibility.set_seed()
rnn = RNN(params)
m, errs = rnn.run_model(rnn_dat2)

In [ ]:
import importlib
importlib.reload(moisture_rnn)
from moisture_rnn import RNN_LSTM

In [ ]:
with open("params.yaml") as file:
    params = yaml.safe_load(file)["lstm"]

rnn_dat2 = create_rnn_data2(train[case],params)
params

In [ ]:
params.update({
    'learning_rate': 0.000001,
    'epochs': 10,
    'clipvalue':1.0
})

In [ ]:
reproducibility.set_seed()
lstm = RNN_LSTM(params)
m, errs = lstm.run_model(rnn_dat2)